<a href="https://colab.research.google.com/github/mvish7/PyTorch/blob/master/NN_in_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
trainset=torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',
                                           train=True, 
                                           download=True,
               transform=transforms.Compose([transforms.ToTensor()]))

  0%|          | 0/26421880 [00:00<?, ?it/s]

26427392it [00:00, 71735944.29it/s]                              


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 520314.35it/s]
  5%|▍         | 212992/4422102 [00:00<00:02, 1945023.53it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:00, 25455184.76it/s]                           
8192it [00:00, 158012.13it/s]


Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    
  def forward(self,t):
    #implementing fwd pass
    
    # input layer
    t=t #returning the tensor as it as, no need of processing
    
    # hidden layer 1 
    t=self.conv1(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    
    #hidden layer 2
    t=self.conv2(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden layer 3
    t=t.reshape(-1,12*4*4)  #doing the flattening operation before passing the tensor from conv layer to the linear layer
    t=self.fc1(t)
    t=F.relu(t)
    
    #hidden layer 4
    t=self.fc2(t)
    t=F.relu(t)
    
    #output layer
    t=self.out(t)
    #t=F.softmax(t, dim=1)  no need to use softmax here, coz we are using cross entropy as our loss function and it has built in softmax.
    
    
    return t
  
      

In [0]:
network=Network()

fetching one image from training set

In [0]:
sample_img=next(iter(trainset))

In [6]:
image, label=sample_img
image.shape

torch.Size([1, 28, 28])

as torch requires the ip tensor in the form of [batch_size, ip channels, W, H] we have to change the shape of this tensor by unsqueez operation

In [7]:
image.unsqueeze(0).shape #now we can send this in.

torch.Size([1, 1, 28, 28])

let's get a rough prediction from image (forward pass)

In [0]:
pred = network(image.unsqueeze(0))

In [9]:
pred.shape


torch.Size([1, 10])

In [10]:
pred

tensor([[-0.0075, -0.1422, -0.1128, -0.0014,  0.0011,  0.1126,  0.0267,  0.0199,
         -0.0983, -0.0984]], grad_fn=<AddmmBackward>)

In [11]:

label

9

getting the probabilities of every prediction

In [12]:
F.softmax(pred, dim=1) #why dim=1? last axis of the temsor has the numbers other axes has the arrays

tensor([[0.1020, 0.0891, 0.0918, 0.1026, 0.1029, 0.1150, 0.1055, 0.1048, 0.0931,
         0.0931]], grad_fn=<SoftmaxBackward>)

now sending batch of images to n/w as ip


---


creating the data loader instance and setting batch size to 10

In [0]:
data_loader=torch.utils.data.DataLoader(trainset,batch_size=10)

In [0]:
batch= next(iter(data_loader))

batch of 10 images is in the shape of [batch_size, ip channels, h,w]

In [0]:
images, labels=batch

In [19]:
images.shape #10 images with 1 color channel with 28*28 resolution

torch.Size([10, 1, 28, 28])

In [20]:
labels.shape

torch.Size([10])

In [0]:
preds=network(images)

In [22]:
preds.shape

torch.Size([10, 10])

as we have 10 images we have 10 prediction classes for 10 images so o/p size is 10*10

In [23]:
preds.argmax(dim=1)

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [24]:
labels


tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

lets try to find number of correct predictions


> we will equate the number labels with pred.argmax, where there is a match between predictions and true match we should get one( fashion mnist dataset has a predefined order to labels and correcponding clothing)



In [25]:
preds.argmax(dim=1).eq(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=torch.uint8)

In [26]:
preds.argmax(dim=1).eq(labels).sum() 

tensor(2)

so 2 predictions out of 10 were correct


---

now create a function which tells us the number of correct prediction when we give predictions and labels as ip

In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()
   